In [ ]:
import redis as r

REDIS_PORT = 6379
r = r.Redis(host='localhost',
                port=REDIS_PORT,
                decode_responses=True)

In [18]:
r.scan(match="*",
       _type="ZSET")[1]

['ED_0:PL_0-B:SALA_23',
 'ED_3:PL_3-A:SALA_35',
 'ED_1:PL_1-A:SALA_20',
 'ED_2:PL_4-A:SALA_60',
 'ED_1:PL_1-B:SALA_35']